In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
matches_df = pd.read_csv("../Data/matches.csv")
events_df = pd.read_csv("../Data/events.csv")
tables_df = pd.read_csv("../Data/all_tables.csv")
players_df = pd.read_csv("../Data/all_players.csv")
goalies_df = pd.read_csv("../Data/all_goalkeepers.csv")

C:\Users\andyr\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (11,14,50,102,123,125,129,131,133,135,137,139,141,143,145,147,149,157,158,160,161,163,165,167,169,171,173,184,186,187,205,206) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


# Prepare matches data

In [3]:
matches_df.columns.tolist()

['id',
 'home',
 'away',
 'date',
 'year',
 'time (utc)',
 'attendance',
 'venue',
 'league',
 'part_of_competition',
 'game_status',
 'shootout',
 'home_score',
 'away_score',
 'referee',
 'home_goal_minutes',
 'home_goal_scorers',
 'away_goal_minutes',
 'away_goal_scorers',
 'home_starting_1_num',
 'home_starting_1',
 'home_starting_2_num',
 'home_starting_2',
 'home_starting_3_num',
 'home_starting_3',
 'home_starting_4_num',
 'home_starting_4',
 'home_starting_5_num',
 'home_starting_5',
 'home_starting_6_num',
 'home_starting_6',
 'home_starting_7_num',
 'home_starting_7',
 'home_starting_8_num',
 'home_starting_8',
 'home_starting_9_num',
 'home_starting_9',
 'home_starting_10_num',
 'home_starting_10',
 'home_starting_11_num',
 'home_starting_11',
 'home_bench_1_num',
 'home_bench_1',
 'home_bench_1_minute',
 'home_bench_2_num',
 'home_bench_2',
 'home_bench_2_minute',
 'home_bench_3_num',
 'home_bench_3',
 'home_bench_3_minute',
 'home_bench_4_num',
 'home_bench_4',
 'home_benc

#Drop columns that will not be used.

In [4]:
matches = matches_df.drop(['time (utc)', 'venue', 'league', 'part_of_competition', 'shootout', 'game_status'], axis=1)
matches.head()

,id,home,away,date,year,attendance,home_score,away_score,referee,home_goal_minutes,...,away_offsides,home_wonCorners,away_wonCorners,home_saves,away_saves,away_bench_14_num,away_bench_14,away_bench_14_minute,home_formation,away_formation
0,NaN,New England,San Jose,7/31/1996,1996,"12,871",2,0,Ruben Rodhas,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Dallas,Colorado,6/15/1996,1996,"9,704",1,1,Rich Grady,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,Colorado,D.C. United,8/29/1996,1996,"6,368",1,2,Ali Saheli,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,LA Galaxy,New England,8/8/1996,1996,"10,251",1,0,Esse Baharmast,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,New England,D.C. United,7/20/1996,1996,"18,347",2,0,Kevin Stott,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#Drop any "starting" and "bench" columns.

In [5]:
matches2 = matches[[item for item in matches.columns.tolist() if 'starting' not in item and 'bench' not in item]]
matches2.head()

,id,home,away,date,year,attendance,home_score,away_score,referee,home_goal_minutes,...,home_redCards,away_redCards,home_offsides,away_offsides,home_wonCorners,away_wonCorners,home_saves,away_saves,home_formation,away_formation
0,NaN,New England,San Jose,7/31/1996,1996,"12,871",2,0,Ruben Rodhas,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Dallas,Colorado,6/15/1996,1996,"9,704",1,1,Rich Grady,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,Colorado,D.C. United,8/29/1996,1996,"6,368",1,2,Ali Saheli,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,LA Galaxy,New England,8/8/1996,1996,"10,251",1,0,Esse Baharmast,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,New England,D.C. United,7/20/1996,1996,"18,347",2,0,Kevin Stott,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
matches2['attendance'] = matches2['attendance'].str.replace(',', '')
matches2

<ipython-input-6-72651b259805>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matches2['attendance'] = matches2['attendance'].str.replace(',', '')


,id,home,away,date,year,attendance,home_score,away_score,referee,home_goal_minutes,...,home_redCards,away_redCards,home_offsides,away_offsides,home_wonCorners,away_wonCorners,home_saves,away_saves,home_formation,away_formation
0,NaN,New England,San Jose,7/31/1996,1996,12871,2,0,Ruben Rodhas,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Dallas,Colorado,6/15/1996,1996,9704,1,1,Rich Grady,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,Colorado,D.C. United,8/29/1996,1996,6368,1,2,Ali Saheli,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,LA Galaxy,New England,8/8/1996,1996,10251,1,0,Esse Baharmast,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,New England,D.C. United,7/20/1996,1996,18347,2,0,Kevin Stott,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6990,598143.0,LA Galaxy,Minnesota United FC,"Sunday, November 7",2021,23803,3,3,NaN,45'+1':51': 75',...,0.0,0.0,1.0,1.0,5.0,7.0,1.0,7.0,4-4-2,4-2-3-1
6991,598142.0,Portland Timbers,Austin FC,"Sunday, November 7",2021,23127,3,0,NaN,17':45':53',...,0.0,0.0,0.0,2.0,5.0,6.0,4.0,6.0,4-2-3-1,4-2-3-1
6992,598141.0,San Jose Earthquakes,FC Dallas,"Sunday, November 7",2021,NaN,1,1,NaN,34',...,0.0,0.0,1.0,2.0,10.0,0.0,0.0,6.0,4-2-3-1,4-2-3-1
6993,598145.0,Sporting Kansas City,Real Salt Lake,"Sunday, November 7",2021,19022,0,1,NaN,NaN,...,0.0,0.0,1.0,1.0,1.0,0.0,2.0,4.0,4-3-3,4-4-2


In [7]:
matches2[['year','attendance']] = matches2[['year','attendance']].astype(float).astype('Int32')

C:\Users\andyr\anaconda3\lib\site-packages\pandas\core\frame.py:3191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


#Average attendance per season total and per team. Make chart with trendline for season totals.

In [20]:
matches2.groupby(['year'])['attendance'].mean()

year
1996    17431.841808
1997    14711.450867
1998    14552.742718
1999    14420.793269
2000    11957.529412
2001            <NA>
2002            <NA>
2003            <NA>
2004            <NA>
2005            <NA>
2006            <NA>
2007            <NA>
2008    16310.681818
2009    16120.959276
2010         36300.0
2011            <NA>
2012    18964.532738
2013    18785.649852
2014     19351.81791
2015    21628.814607
2016    21982.481793
2017    22311.253197
2018    22174.053922
2019    21703.230404
2020    11366.272727
2021    17033.544959
Name: attendance, dtype: Float64

#Home goals vs away goals + chart

# Prepare events data to join with matches2

#Viewing single game events

In [21]:
events_df.head()

,id,Time,Event
0,14876,-,no commentary
1,14877,-,no commentary
2,14879,-,no commentary
3,14878,-,no commentary
4,14880,-,no commentary


In [22]:
single_game_events = events_df[ events_df['id']==598144 ]
print(len(single_game_events))
single_game_events.head()

85


,id,Time,Event
473245,598144,-,Lineups are announced and players are warming up.
473246,598144,-,First Half begins.
473247,598144,1',"Corner, Vancouver Whitecaps FC. Conceded by J..."
473248,598144,2',Attempt missed. Cristián Gutiérrez (Vancouver ...
473249,598144,2',"Corner, Vancouver Whitecaps FC. Conceded by Y..."


#Filterin goals scored, minute scored, and score per team

In [23]:
single_game_events[single_game_events['Event'].str.contains('Goal!')]

,id,Time,Event
473254,598144,8',"Goal! Vancouver Whitecaps FC 0, Seattle Sound..."
473263,598144,20',"Goal! Vancouver Whitecaps FC 1, Seattle Sound..."


#Make loop for all goals, yellow cards, red cards. Merge on "id." 

#Get 15 minute window of first and second half with the most goals scored.

#Get 15 minute window with the most red and yellow cards in second half.

#Get average time for red cards.

#Find crrelation between red cards and penalty shots. Pie chart.

#Find correlation between yellow cards and teams trailing. (Maybe scatter plots?)

# Prepare tables_df

In [ ]:
tables_df.head

#Filter 2020 season

In [ ]:
table_2020 = tables_df[tables_df['Year']==2020 ].reset_index()
table_2020

#Web scraping -- most points per season -- most points total -- least points per season -- average position per team

#Most points in debut season -- least points in debut season -- most goals in debut season -- least goals against in debut season

# Web scraping -- all time scoring leaders -- most goals per season(player) -- most assists

# Web scraping -- all time goalie leaders -- least goals per season -- most saves

# Web scraping -- fouls committed/fouls suffered(player/team) -- penalty kicks given vs penalty kicks caused -- 